<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Herrera Suárez
- Nombre de alumno 2: Hecmar Taucare Reyes


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/Joaquin-HS/MDS7202)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
# !pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
import pandas as pd

data = pd.read_csv('sales.csv')

data.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [2]:
data.columns

Index(['id', 'date', 'city', 'lat', 'long', 'pop', 'shop', 'brand',
       'container', 'capacity', 'price', 'quantity'],
      dtype='object')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7456 entries, 0 to 7455
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         7456 non-null   int64  
 1   date       7456 non-null   object 
 2   city       7456 non-null   object 
 3   lat        7456 non-null   float64
 4   long       7456 non-null   float64
 5   pop        7456 non-null   int64  
 6   shop       7456 non-null   object 
 7   brand      7456 non-null   object 
 8   container  7456 non-null   object 
 9   capacity   7456 non-null   object 
 10  price      7456 non-null   float64
 11  quantity   7456 non-null   int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 699.1+ KB


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [4]:
from sklearn import set_config
set_config(transform_output="pandas")

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib

In [5]:
# 1. Se separan los datos en train (70%), validation (20%) y test (10%)
X = data.drop(columns=['quantity'])  # Características
y = data['quantity']  # Variable objetivo

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

In [6]:
# 2. Se implementa un FunctionTransformer para extraer el día, mes y año de la variable 'date'.
# Se guardan las variables en el formato categorical de pandas.
def extract_date_features(df):
    df = df.copy()
    df['year'] = pd.Categorical(pd.to_datetime(df['date'], format='%d/%m/%y').dt.year)
    df['month'] = pd.Categorical(pd.to_datetime(df['date'], format='%d/%m/%y').dt.month)
    df['day'] = pd.Categorical(pd.to_datetime(df['date'], format='%d/%m/%y').dt.day)
    return df.drop(columns='date')

date_transformer = FunctionTransformer(extract_date_features)

In [7]:
# 3. Se implementa un ColumnTransformer para procesar variables numéricas y categóricas
numeric_features = ['lat', 'long', 'pop', 'price']  # 'quantity' es la variable objetivo
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'year', 'month', 'day']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)
preprocessor.set_output(transform='pandas')

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['lat', 'long', 'pop', 'price']),
                                ('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['city', 'shop', 'brand', 'container',
                                  'capacity', 'year', 'month', 'day'])])

In [8]:
# 4. Se crea el Pipeline con DummyRegressor para generar predicciones basadas en la media
dummy_pipeline = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

In [9]:
# 5. Se entrena el pipeline y se calcula el MAE en datos de validación
dummy_pipeline.fit(X_train, y_train)
y_val_pred_dummy = dummy_pipeline.predict(X_val)
mae_dummy = mean_absolute_error(y_val, y_val_pred_dummy)
print(f"MAE DummyRegressor: {mae_dummy:.2f}")

MAE DummyRegressor: 13308.13


El MAE es la media del error absoluto entre las predicciones y los valores reales. Para el negocio indica, en promedio, cuántas unidades se están errando al predecir las ventas.

In [10]:
# 6. Se reentrena el Pipeline con XGBRegressor
xgb_pipeline = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())  # Parámetros por defecto
])

xgb_pipeline.fit(X_train, y_train)
y_val_pred_xgb = xgb_pipeline.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f"MAE XGBRegressor: {mae_xgb:.2f}")

MAE XGBRegressor: 2423.12


El MAE del XGB es menor que el del DummyRegressor, lo que significa que el modelo es mejor en la predicción de ventas.

In [11]:
# 7. Se guardan ambos modelos en archivos .pkl
joblib.dump(dummy_pipeline, 'dummy_model.pkl')
joblib.dump(xgb_pipeline, 'xgb_model.pkl')

['xgb_model.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [13]:
# 1. Se vuelve a entrenar el pipeline con XGBRegressor, forzando una relación monótona negativa entre 'price' y 'quantity'

# Se obtienen los nombres de las columnas después del preprocesamiento (numéricas y las codificadas por OneHotEncoder)
X_train_transformed = date_transformer.transform(X_train)
preprocessor.fit(X_train_transformed)
feature_names = preprocessor.get_feature_names_out()

# Se identifica el índice de la columna que corresponde a 'price'
price_index = list(feature_names).index('num__price')

# Se crea la lista de restricciones de monotonicidad (una entrada por cada característica)
# Se fuerza una relación monótona negativa en la columna del precio (-1)
# El resto de las columnas no tienen restricciones (0)
monotonic_constraints = [0] * len(feature_names)  # Se inicializa con 0 (sin restricción)
monotonic_constraints[price_index] = -1  # Relación monótona negativa con el precio
monotonic_constraints = "(" + ",".join(map(str, monotonic_constraints)) + ")"  # Lista de restricciones de monotonicidad en una cadena que XGBoost puede interpretar

# Se entrena el Pipeline con XGBRegressor aplicando la restricción de monotonicidad
xgb_monotonic_pipeline = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=monotonic_constraints))
])

xgb_monotonic_pipeline.fit(X_train, y_train)

Pipeline(steps=[('date_transformer',
                 FunctionTransformer(func=<function extract_date_features at 0x00000212F3562A20>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity',
                                                   'year', 'month', 'day']...
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints='(0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [15]:
# 2. Se reporta el MAE sobre el conjunto de validación

# Se predice en el conjunto de validación
y_val_pred_xgb_monotonic = xgb_monotonic_pipeline.predict(X_val)

# Se calcula el MAE con la restricción
mae_xgb_monotonic = mean_absolute_error(y_val, y_val_pred_xgb_monotonic)
print(f"MAE XGBRegressor con restricción monótona: {mae_xgb_monotonic:.2f}")

MAE XGBRegressor con restricción monótona: 2480.25


Como se puede ver, el error aumentó levemente con respecto al modelo anterior sin restricciones. Así, la hipótesis del amigo no logra ser suficiente para mejorar el modelo.

In [16]:
# 4. Se guardan el modelo en un archivo .pkl
joblib.dump(xgb_monotonic_pipeline, 'xgb_monotonic_model.pkl')

['xgb_monotonic_model.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [23]:
# 1. Se implementa la función objective() para minimizar el MAE en el conjunto de validación

import optuna
from optuna.samplers import TPESampler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Se fija la semilla para garantizar la reproducibilidad
semilla = 42
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # Se optimizan los hiperparámetros de XGBRegressor
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    
    # Se optimiza el hiperparámetro de OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    # Se construye el pipeline con preprocesamiento y modelo
    xgb_pipeline = Pipeline(steps=[
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor.set_params(
            cat__min_frequency=min_frequency
        )),
        ('regressor', XGBRegressor(
            random_state=semilla,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            # monotone_constraints=monotonic_constraints     # No se incluye la restricción monótona por lo visto en la sección anterior
        ))
    ])
    
    # Se entrena el pipeline
    xgb_pipeline.fit(X_train, y_train)
    
    # Se predice en el conjunto de validación
    y_pred = xgb_pipeline.predict(X_val)
    
    # Se calcula el MAE
    mae = mean_absolute_error(y_val, y_pred)
    
    # Se almacena el mejor pipeline
    trial.set_user_attr("best_pipeline", xgb_pipeline)
    
    return mae

# Se configura el estudio de optuna con TPESampler
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=semilla))

In [24]:
# 2. Se fija el tiempo de entrenamiento en 5 minutos

import time

# Callback para detener la optimización después de 5 minutos
class TimeLimitCallback:
    def __init__(self, time_limit_seconds):
        self.time_limit_seconds = time_limit_seconds
        self.start_time = time.time()

    def __call__(self, study, trial):
        elapsed_time = time.time() - self.start_time
        if elapsed_time > self.time_limit_seconds:
            raise optuna.exceptions.OptunaError('Tiempo límite alcanzado.')

# Se limita la optimización a 5 minutos (300 segundos)
time_limit = 300
time_limit_callback = TimeLimitCallback(time_limit)

# Se ejecuta la optimización con el tiempo límite y se reporta el mejor trial
study.optimize(objective, n_trials=100, callbacks=[time_limit_callback])

In [26]:
# 3. Se reportan los resultados obtenidos

# Número de trials, MAE y mejores hiperparámetros
print(f"Número de trials: {len(study.trials)}")
print(f"Mejor MAE: {study.best_value:.2f}")
print(f"Mejores hiperparámetros: {study.best_trial.params}")

# Se almacena el mejor pipeline
best_pipeline = study.best_trial.user_attrs["best_pipeline"]

Número de trials: 100
Mejor MAE: 1945.87
Mejores hiperparámetros: {'learning_rate': 0.09798314261891256, 'n_estimators': 822, 'max_depth': 9, 'max_leaves': 96, 'min_child_weight': 5, 'reg_alpha': 0.004791828514001553, 'reg_lambda': 0.8873929584923299, 'min_frequency': 0.07438776260288232}


Se observa que el MAE del mejor modelo se redujo considerablemente con respecto al modelo de referencia (el que tenía parámetros por defecto), al pasar de `2423.12` a `1945.87`. Esto se debe a que el primer modelo no estaba calibrado (hiperparámetros) con respecto a los datos que se tienen.

A continuación se explican cada uno de los hiperparámetros analizados:

- `learning_rate`: corresponde al tamaño de los saltos dados al minimizar el error. Un rango entre 0.001 y 0.1 es adecuado para encontrar un buen *trade-off* entre convergencia rápida y evitar sobreajuste.

- `n_estimators`: es la cantidad de árboles que usa el modelo. El rango entre 50 y 1000 permite explorar desde modelos pequeños hasta grandes con muchos árboles.

- `max_depth`: corresponde a la profundidad máxima de cada árbol. Un rango entre 3 y 10 permite controlar qué tan complejo es el modelo.

- `max_leaves`: es la cantidad máxima de hojas por árbol. El rango de 0 a 100 permite controlar la complejidad y el sobreajuste.

- `min_child_weight`: controla el peso mínimo de la sumatoria de observaciones que se necesitan en un nodo para que se divida. El rango de 1 a 5 aborda distintas configuraciones de tamaño mínimo de nodos y hojas, ayudando a reducir sobreajuste.

- `reg_alpha`: corresponde a la regularización L1, que agrega una penalización proporcional a la suma absoluta de los coeficientes de los parámetros. El rango entre 0 y 1 (con valores flotantes) aborda distintos niveles de penalización, regulando así el desempeño y la simplicidad del modelo.

- `reg_lambda`: corresponde a la regularización L2, que agrega una penalización proporcional al cuadrado de los coeficientes de los parámetros. Se explica de igual forma que para el parámetro anterior, pero ahora con la norma L2.

- `min_frequency`: en OneHotEncoder controla la frecuencia mínima para codificar una categoría. El rango entre 0 y 1 (con valores flotantes) permite revisar distintas configuraciones de proporción de la cantidad total de observaciones que delimitan si cierta categoría se categoriza como "otro" (si no cumple con la frecuencia mínima) o no.

In [28]:
# 5. Se guarda el mejor modelo en un archivo .pkl
joblib.dump(best_pipeline, 'mejor_modelo.pkl')

['mejor_modelo.pkl']

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
#!pip install optuna-integration[xgboost]

*Prunning* es una técnica utilizada en la optimización de modelos que detiene de manera temprana las ejecuciones de configuraciones de hiperparámetros que no muestren mejoras significativas en cuanto a desempeño. De esta forma, se monitorea el funcionamiento del modelo en las primeras etapas del entrenamiento, interrumpiendo dicho *trial* si no es prometedor. Esto ahorra tiempo y recursos, ya que evita entrenar completamente modelos que no funcionarán. Además, permite explorar más combinaciones de hiperparámetros en la misma búsqueda.

In [64]:
# 2. Se redefine la función objective() incluyendo Prunning

import optuna
from optuna.samplers import TPESampler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from optuna.integration import XGBoostPruningCallback

# Se fija la semilla para garantizar la reproducibilidad
semilla = 42
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # Se optimizan los hiperparámetros de XGBRegressor
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    
    # Se optimiza el hiperparámetro de OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    
    # Para permitir prunning se deben utilizar argumentos que no están disponibles en el
    # ajuste a través de un pipeline. Se hará el ajuste por separado.
    

    # Se ajusta el preprocesador
    preprocessor.set_params(cat__min_frequency=min_frequency)
    
    # Preprocesamiento de los datos
    X_train_transformed = date_transformer.transform(X_train)
    X_train_processed = preprocessor.fit_transform(X_train_transformed)

    X_val_transformed = date_transformer.transform(X_val)
    X_val_processed = preprocessor.transform(X_val_transformed)

    # Callback de prunning para XGBoost
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-mae")  # Monitorea el MAE en el conjunto de validación

    # Modelo de XGBoost
    model = XGBRegressor(
        random_state=semilla,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        # monotone_constraints=monotonic_constraints     # No se incluye la restricción monótona por lo visto en la sección anterior
        early_stopping_rounds=20,  # Si no hay mejoras en 20 iteraciones, se detiene el entrenamiento
        callbacks=[pruning_callback],
        eval_metric='mae'
    )

    # Se entrena el modelo
    model.fit(
        X_train_processed, y_train,
        eval_set=[(X_val_processed, y_val)],
        verbose=False
    )

    # Se predice en el conjunto de validación
    y_pred = model.predict(X_val_processed)
    
    # Se calcula el MAE
    mae = mean_absolute_error(y_val, y_pred)
    
    # Se almacena el mejor modelo
    trial.set_user_attr("best_pipeline", model)
    
    return mae

# Se configura el estudio de optuna con TPESampler
study_prun = optuna.create_study(direction='minimize', sampler=TPESampler(seed=semilla))

In [65]:
# 3. Se fija el tiempo de entrenamiento en 5 minutos

import time

# Callback para detener la optimización después de 5 minutos
class TimeLimitCallback:
    def __init__(self, time_limit_seconds):
        self.time_limit_seconds = time_limit_seconds
        self.start_time = time.time()

    def __call__(self, study, trial):
        elapsed_time = time.time() - self.start_time
        if elapsed_time > self.time_limit_seconds:
            raise optuna.exceptions.OptunaError('Tiempo límite alcanzado.')

# Se limita la optimización a 5 minutos (300 segundos)
time_limit = 300
time_limit_callback = TimeLimitCallback(time_limit)

# Se ejecuta la optimización con el tiempo límite y se reporta el mejor trial
study_prun.optimize(objective, n_trials=100, callbacks=[time_limit_callback], show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

In [69]:
# 4. Se reportan los resultados obtenidos

# Número de trials, MAE y mejores hiperparámetros
print(f"Número de trials: {len(study_prun.trials)}")
print(f"Mejor MAE: {study_prun.best_value:.2f}")
print(f"Mejores hiperparámetros: {study_prun.best_trial.params}")

# Se almacena el mejor pipeline
best_model_prun = study_prun.best_trial.user_attrs["best_pipeline"]

Número de trials: 100
Mejor MAE: 1931.04
Mejores hiperparámetros: {'learning_rate': 0.09098235113036565, 'n_estimators': 621, 'max_depth': 8, 'max_leaves': 85, 'min_child_weight': 3, 'reg_alpha': 0.7648467819107042, 'reg_lambda': 0.26476814522223974, 'min_frequency': 0.03224486951465588}


Se observa que el MAE se redujo levemente con respecto al mejor modelo obtenido previamente, al pasar de `1945.87` a `1931.04`. Esto se debe a que, al añadir prunning, se revisaron más combinaciones de hiperparámetros de valor, logrando así encontrar uno que calibra mejor con los datos que se tienen. Ahora bien, se destaca que el tiempo de ejecución aumentó considerablemente, lo que se puede deber a que no se realizó la implementación con la arquitectura de pipelines de scikit-learn, lo que hizo que, por ejemplo, se realizaran transformaciones repetidas de los datos de entrada.

In [67]:
# 5. Se guarda el mejor modelo en un archivo .pkl
joblib.dump(best_model_prun, 'mejor_modelo_prunning.pkl')

['mejor_modelo_prunning.pkl']

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [70]:
# 1. Gráfico de historial de optimización

import optuna

fig = optuna.visualization.plot_optimization_history(study_prun)
fig.show()

En este gráfico se aprecia que se empiezan a observar mejoras notables en los resultados a partir del *trial* 13, donde el MAE llega a 2158.69. Finalmente, es en el trial 54 donde se obtiene el mejor MAE: 1931.04. Se destaca que a partir del *trial* 13 los valores de MAE se estabilizan, con leves mejoras, hasta llegar al mínimo encontrado.

In [71]:
# 2. Gráfico de coordenadas paralelas

fig = optuna.visualization.plot_parallel_coordinate(study_prun)
fig.show()

Teniendo en cuenta que el gráfico de coordenadas paralelas muestra cómo los diferentes hiperparámetros influyen en el valor objetivo (MAE en este caso), y que cada línea representa un conjunto de hiperparámetros evaluado durante la optimización (líneas oscuras representan mejores desempeños, con valores objetivos de MAE más bajos), se destacan las siguientes tendencias:

- `learning rate`: valores cercanos a 0.08 producen mejores desempeños para el modelo. Esto genera que el modelo ajusta los pesos de manera más agresiva, reduciendo el error rápidamente, pero pudiendo producir pérdidas en precisión y una menor estabilidad en cuanto a los patrones aprendidos.

- `n_estimators`, `max_depth`, `max_leaves` y `min_child_weight`: estas variables no llegan a ser del todo determinantes, ya que con diferentes valores dentro de cada rango de búsqueda se pueden obtener buenos desempeños del modelo.

- `reg_alpha` y `reg_lambda`: valores entre 0.6 y 0.8, y entorno a 0.2 o 0.8 generan los mejores resultados, respectivamente. Esto indica que en general los modelos más penalizados (que empujan a más coeficientes a ser cero) tienen un mejor desempeño.

- `min_frequency`: valores cercanos a 0.1 e incluso menores son los que producen mejores desempeños para el modelo. Esto implica que incluso categorías que tienen poca representación serán diferenciadas, aumentando así la dimensionalidad de los datos. De esta forma, por un parte se captura información de eventos poco frecuentes (algo beneficioso si aportan valor predictivo), pero por otra aumenta la complejidad computacional y sobre todo el riesgo de sobreajuste, generando que el modelo puede no generalizar bien.

In [72]:
# 3. Gráfico de importancia de hiperparámetros

fig = optuna.visualization.plot_param_importances(study_prun)
fig.show()

Los hiperparámetros con mayor importancia para la optimización del modelo son los siguientes:

- `min_frequency` = 0.83. Este hiperparámetro tiene una influencia significativamente mayor que el resto. Esto indica que la manera en la que se manejan las categorías poco frecuentes tiene un impacto crucial en el desempeño del modelo.

- `learning_rate` = 0.08. Con una notable menor importancia que `min_frequency`, la tasa de aprendizaje también tiene un papel relevante en la convergencia del modelo.

- `reg_alpha` = 0.03. La regularización L1 tiene un leve impacto, ayudando a controlar la complejidad del modelo. Por su parte, `reg_lambda` y `n_estimators` (con 0.02 cada uno) también tienen una relevancia leve, pero siguen contribuyendo al equilibrio entre sobreajuste y rendimiento.

- Los demás hiperparámetros (`max_leaves`, `max_depth`, y `min_child_weight`) tienen una importancia inferior a 0.01, lo que indica que su ajuste tiene poco impacto directo en el desempeño del modelo.

Nótese que estos resultados coinciden con el análisis previo sobre el gráfico de coordenadas paralelas.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [75]:
# 1.

import pandas as pd

# Resultados de MAE para cada modelo
resultados = {
    "Modelo": [
        "Baseline", 
        "XGBoost", 
        "XGBoost con Constraints", 
        "Optuna", 
        "Optuna con Prunning"
    ],
    "MAE": [
        f"{mae_dummy:.2f}",  # Baseline
        f"{mae_xgb:.2f}",  # XGBoost
        f"{mae_xgb_monotonic:.2f}",  # XGBoost con Constraints
        f"{study.best_value:.2f}",  # Optuna
        f"{study_prun.best_value:.2f}"  # Optuna con Prunning
    ]
}
df_resultados = pd.DataFrame(resultados)
df_resultados

,Modelo,MAE
0,Baseline,13308.13
1,XGBoost,2423.12
2,XGBoost con Constraints,2480.25
3,Optuna,1945.87
4,Optuna con Prunning,1931.04


De la tabla se aprecia que el modelo obtenido mediante Optuna con Prunning fue el que obtuvo el mejor rendimiento (menor error).

In [81]:
# 3.

import joblib
from sklearn.metrics import mean_absolute_error

# Preprocesamiento de los datos
X_test_transformed = date_transformer.transform(X_test)
X_test_processed = preprocessor.fit_transform(X_test_transformed)

# Se carga el modelo
modelo_cargado = joblib.load("mejor_modelo_prunning.pkl")

# Se predice sobre el conjunto de test
y_test_pred = modelo_cargado.predict(X_test_processed)

# Se calcula el MAE en el conjunto de test
mae_test = mean_absolute_error(y_test, y_test_pred)
print(f"MAE en el conjunto de test: {mae_test:.2f}")

MAE en el conjunto de test: 3724.61


Como se puede ver, si existe una diferencia entre el MAE de validación y MAE de testeo. Esto ocurre porque el modelo se ajusta según los datos de validación, pudiendo no generalizar bien a los datos de test (algo que se mencionó previamente). Además, las distribuciones de los conjuntos de validación y test podrían ser diferentes, lo que puede causar variaciones en el rendimiento.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>